In [2]:
import numpy as np
import cv2
import os

import random
import torch
from torch.autograd import Variable
output_path = "mnist_train_images/"
train_path = "mnist_train.csv"
test_path = "mnist_test.csv"

In [3]:
def convert_dataset(train_path):
    dataset = np.genfromtxt(train_path, delimiter=',',skip_header=1)
    labels = dataset[:, 0].astype(np.uint8)
    values = dataset[:,1:].astype(np.uint8)
    images = np.reshape(values, (-1,28,28))
    return images,labels

In [4]:
# dataset = np.genfromtxt("mnist_train.csv", delimiter=',', skip_header=1)
# labels = dataset[:, 0].astype(np.uint8)
# values = dataset[:,1:].astype(np.uint8)
# images = np.reshape(values, (-1,28,28))
# images.shape

images, labels = convert_dataset(train_path)

In [5]:
test_images, test_labels = convert_dataset(test_path)

In [6]:
learning_rate = 0.01
batch_size = 200
epochs = 10
log_interval = 10

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)


In [9]:
net = Net()
net

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)

In [10]:
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum=0.9)
criterion = nn.NLLLoss()

In [11]:
def rotate(image, label):
        degree = random.randint(1,180)
        if (label == 6 or label == 9) and degree == 90:
            while degree != 90:
                degree = random.randint(1, 180)
        rot_img = np.uint8(np.zeros(image.shape))
        height, width = rot_img.shape
        midx,midy = (width//2, height//2)
        for i in range(rot_img.shape[0]):
            for j in range(rot_img.shape[1]):
                x= (i-midx)*np.cos(degree)+(j-midy)*np.sin(degree)
                y= -(i-midx)*np.sin(degree)+(j-midy)*np.cos(degree)
                x=round(x)+midx
                y=round(y)+midy
                if (x>=0 and y>=0 and x<image.shape[0] and  y<image.shape[1]):
                    rot_img[i,j] = image[x,y]
        return rot_img

In [12]:
def noise(im):
    noize = np.zeros(im.shape,np.uint8)
    cv2.randn(noize,0,50)
    n_im = cv2.add(im,noize)
    return n_im

In [13]:
def normalize(im):
    imin = float(im.min())
    imax = float(im.max())
    return (im - imin)/(imax - imin)

In [14]:
def conf(pred, target):
    confusion_vector = pred/target
    tp = torch.sum(confusion_vector==1).item()
    fp = torch.sum(confusion_vector == float('inf')).item()
    tn = torch.sum(torch.isnan(confusion_vector)).item()
    fn = torch.sum(confusion_vector == 0).item()
    return tp,fp,tn,fn

In [15]:
def saveModel():
    path = './net.pth'
    torch.save(net.state_dict(),path)

In [16]:
for epoch in range(epochs):
    for i in range(len(images) // batch_size):
        batch = batch_size * i
        i_batch = images[batch:batch + batch_size]
        l_batch = labels[batch:batch + batch_size]
        for j in range(len(i_batch)):
            i_batch[j] = noise(i_batch[j])
            # i_batch[j] = rotate(i_batch[j], l_batch[j])
        im_batch = torch.from_numpy(normalize(i_batch)).float()
        lb_batch = torch.from_numpy(l_batch)

        data, target = Variable(im_batch), Variable(lb_batch)
        data = data.view(-1,28*28)
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(images),
                       100. * batch / len(images), loss.data))
    saveModel()

/tmp/ipykernel_14760/3114604308.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.308064
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.288590
Train Epoch: 0 [4000/60000 (7%)]	Loss: 2.270425
Train Epoch: 0 [6000/60000 (10%)]	Loss: 2.220027
Train Epoch: 0 [8000/60000 (13%)]	Loss: 2.159842
Train Epoch: 0 [10000/60000 (17%)]	Loss: 2.093765
Train Epoch: 0 [12000/60000 (20%)]	Loss: 1.949393
Train Epoch: 0 [14000/60000 (23%)]	Loss: 1.756460
Train Epoch: 0 [16000/60000 (27%)]	Loss: 1.549798
Train Epoch: 0 [18000/60000 (30%)]	Loss: 1.192355
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.082516
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.881135
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.676848
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.557882
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.535921
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.775524
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.630645
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.480651
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.491313
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.453419
Train Epoch

In [17]:
norm_test_values = normalize(test_images)

In [18]:
test_loss = 0
correct = 0
tp= fp = tn = fn = 0
for i in range(len(norm_test_values)//batch_size):
    batch = batch_size * i
    im_batch = torch.from_numpy(norm_test_values[batch:batch + batch_size]).float()
    lb_batch = torch.from_numpy(test_labels[batch:batch + batch_size])
    data, target = Variable(im_batch, volatile=True), Variable(lb_batch)
    data = data.view(-1,28*28)
    net_out = net(data)
    test_loss += criterion(net_out, target).data
    pred = net_out.data.max(1)[1]
    correct += pred.eq(target.data).sum()
    tp1,fp1,tn1,fn1 = conf(pred,target.data)
    tp += tp1
    fp += fp1
    tn += tn1
    fn += fn1
test_loss /= len(test_images)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1 = 2*precision * recall/(precision + recall)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_images),
    100. * correct / len(test_images)))
print("Precision %.4f "%precision,'Recall %.4f '%recall, "F1 %.4f "%f1 )


Test set: Average loss: 0.0017, Accuracy: 9239/10000 (92%)

Precision 0.9980  Recall 0.9900  F1 0.9939 


/tmp/ipykernel_14760/1079460992.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(im_batch, volatile=True), Variable(lb_batch)
/tmp/ipykernel_14760/3114604308.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [20]:
def inference(path):
    images1, labels1 = convert_dataset(path)
    arr = []
    norm_images = normalize(images1)
    with torch.no_grad():
        for i in range(len(norm_images)//batch_size):
            batch = i * batch
            data = torch.from_numpy(norm_images[batch:batch + batch_size]).float()
            data = data.view(-1,28*28)
            output = net(data)
            _, predicted = torch.max(output.data, 1)
            arr.append(predicted)
    return arr